<a href="https://colab.research.google.com/github/Kazuky1141/Colab_IA/blob/master/ESIS_Inteligencia_Artificial_Unidad_2_Semana_4b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4b. Despliegue de su modelo
Ahora que tenemos un modelo bien entrenado, es hora de utilizarlo. En este ejercicio, expondremos nuevas imágenes a nuestro modelo y detectaremos las letras correctas del alfabeto del lenguaje de signos. ¡Vamos a empezar!

## 4b.1 Objetivos

* Cargar desde disco un modelo ya entrenado
* Reformatear imágenes para un modelo entrenado con imágenes de formato diferente.
* Realizar inferencias con imágenes nuevas, nunca vistas por el modelo entrenado y evaluar su rendimiento.

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import torchvision.io as tv_io
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

## 4b.2 Cargando el Modelo

Ahora que estamos en un nuevo cuaderno, vamos a cargar el modelo guardado que hemos entrenado. Nuestro guardado del ejercicio anterior creó una carpeta llamada «asl_model». Podemos cargar el modelo seleccionando la misma carpeta.

Dado que nuestro modelo utiliza un [módulo personalizado](https://pytorch.org/tutorials/beginner/examples_nn/two_layer_net_module.html), necesitaremos cargar el código de esa clase. Hemos guardado una copia del código en [utils.py](./uitls.py).

In [ ]:
!apt-get install -y p7zip-full
!wget https://github.com/ichaparroc/IA-EPIS/raw/refs/heads/main/ASL.7z -O ASL.7Z
!7z x ASL.7Z
!rm -f ASL.7Z
!rm -f sign_mnist_train.csv
!rm -f sign_mnist_valid.csv

In [ ]:
from utils import MyConvBlock

Ahora que tenemos una definición para `MyConvBlock`, podemos usar [torch.load](https://pytorch.org/docs/stable/generated/torch.load.html) para cargar un modelo desde una ruta. Podemos usar `map_location` para especificar el dispositivo. Cuando imprimimos el modelo, ¿se ve igual que en el último cuaderno?

In [ ]:
model = torch.load('model.pth', map_location=device, weights_only=False)
model

También podemos verificar si el modelo está en nuestra GPU.

In [ ]:
next(model.parameters()).device

## 4b.3 Preparar una imagen para el modelo

Ahora es el momento de utilizar el modelo para hacer predicciones sobre nuevas imágenes que nunca ha visto antes. Esto también se llama inferencia. Tenemos un conjunto de imágenes en la carpeta `data/asl_images`. Intenta abrirla utilizando el navegador de la izquierda y explora las imágenes.

Te darás cuenta de que las imágenes que tenemos son de mucha mayor resolución que las imágenes de nuestro conjunto de datos. También son en color. Recuerda que nuestras imágenes en el conjunto de datos eran de 28x28 píxeles y en escala de grises. Es importante tener en cuenta que siempre que hacemos predicciones con un modelo, la entrada debe coincidir con la forma de los datos con los que se entrenó el modelo. Para este modelo, el conjunto de datos de entrenamiento tenía la forma: (27455, 28, 28, 1). Esto correspondía a 27455 imágenes de 28 por 28 píxeles cada una con un canal de color (escala de grises).

### 4b.3.1 Mostrar las imágenes

Cuando utilicemos nuestro modelo para hacer predicciones sobre nuevas imágenes, será útil mostrar también la imagen. Para ello, podemos utilizar la biblioteca matplotlib.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image, cmap='gray')

In [ ]:
show_image('b.png')

### 4b.3.2 Escalado de las imágenes

Las imágenes de nuestro conjunto de datos eran de 28x28 píxeles y en escala de grises. Necesitamos asegurarnos de pasar el mismo tamaño e imágenes en escala de grises a nuestro método para la predicción. Hay algunas formas de editar imágenes con Python, pero TorchVision también tiene la función [read_image](https://pytorch.org/vision/stable/generated/torchvision.io.read_image.html). Podemos hacerle saber qué tipo de imagen leer con [ImageReadMode](https://pytorch.org/vision/stable/generated/torchvision.io.ImageReadMode.html#torchvision.io.ImageReadMode).

In [ ]:
image = tv_io.read_image('b.png', tv_io.ImageReadMode.GRAY)
image

Fijémonos en la forma de la imagen.

In [ ]:
image.shape

Esta imagen es mucho más grande que con la que entrenamos. Podemos utilizar [TorchVision](https://pytorch.org/vision/stable/index.html)'s [Transforms](https://pytorch.org/vision/0.9/transforms.html) de nuevo para obtener los datos en la forma que nuestro modelo espera.

Lo haremos:
* Convertiremos la imagen a float con [ToDtype](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.ToDtype.html)
  * Estableceremos `scale` a `True` para convertir de [0, 255] a [0, 1].
* [Redimensionar](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.Resize.html#torchvision.transforms.v2.Resize) la imagen para que sea de 28 x 28 píxeles
* Convertir las imágenes a [Escala de grises](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.Grayscale.html#torchvision.transforms.v2.Grayscale)
  * Este paso no hace nada ya que nuestros modelos ya están en escala de grises, pero lo hemos añadido aquí para mostrar una forma alternativa de obtener imágenes en escala de grises.

In [ ]:
IMG_WIDTH = 28
IMG_HEIGHT = 28

preprocess_trans = transforms.Compose([
    transforms.ToDtype(torch.float32, scale=True), # Converts [0, 255] to [0, 1]
    transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
    transforms.Grayscale()  # From Color to Gray
])

Probemos `preprocess_trans` en una imagen para asegurarnos de que funciona correctamente:

In [ ]:
processed_image = preprocess_trans(image)
processed_image

Los números parecen correctos, pero ¿y la forma?

In [ ]:
processed_image.shape

A continuación, vamos a trazar la imagen para ver si se parece a lo que hemos entrenado.

In [ ]:
plot_image = F.to_pil_image(processed_image)
plt.imshow(plot_image, cmap='gray')

¡Se ve bien! Pasémoslo a nuestro modelo.

### 4b.4 Hacer predicciones

Bien, ¡ahora estamos listos para predecir! Nuestro modelo sigue esperando un lote de imágenes. Si [squeeze](https://pytorch.org/docs/stable/generated/torch.squeeze.htmlhttps://pytorch.org/docs/stable/generated/torch.squeeze.html) elimina dimensiones de 1, [unsqueeze](https://pytorch.org/docs/stable/generated/torch.unsqueeze.htmlhttps://pytorch.org/docs/stable/generated/torch.unsqueeze.html) añade una dimensión de 1 en el índice que especifiquemos. La primera dimensión suele ser la dimensión del lote, por lo que podemos decir `.unsqueeze(0)`.

In [ ]:
batched_image = processed_image.unsqueeze(0)
batched_image.shape

A continuación, debemos asegurarnos de que el tensor de entrada está en el mismo `dispositivo` que el modelo.

In [ ]:
batched_image_gpu = batched_image.to(device)
batched_image_gpu.device

Ahora estamos listos para pasarlo al modelo.

In [ ]:
output = model(batched_image_gpu)
output

### 4b.4.1 Comprender la predicción

Las predicciones tienen el formato de una matriz de 24 longitudes. Cuanto mayor sea el valor, más probable es que la imagen de entrada pertenezca a la clase correspondiente. Hagámoslo un poco más legible. Podemos empezar por encontrar qué elemento de la matriz representa la mayor probabilidad. Esto puede hacerse fácilmente con la biblioteca numpy y la función [argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html).

In [ ]:
prediction = output.argmax(dim=1).item()
prediction

Cada elemento de la matriz de predicción representa una letra posible del alfabeto de la lengua de signos. Recuerda que j y z no son opciones porque implican mover la mano, y sólo estamos tratando con fotos fijas. Creemos una correspondencia entre el índice de la matriz de predicciones y la letra correspondiente.

In [ ]:
# Alphabet does not contain j or z because they require movement
alphabet = "abcdefghiklmnopqrstuvwxy"

Ahora podemos pasar nuestro índice de predicción para encontrar la letra correspondiente.

In [ ]:
alphabet[prediction]

#### Ejercicio: Póngalo todo junto

Pongamos todo en una función para que podamos hacer predicciones sólo a partir del archivo de imagen. Impleméntalo en la función de abajo usando las funciones y pasos anteriores. Si necesitas ayuda, puedes revelar la solución haciendo clic en los tres puntos de abajo.

In [ ]:
def predict_letter(file_path):
    # Show image
    FIXME
    # Load and grayscale image
    image = FIXME
    # Transform image
    image = FIXME
    # Batch image
    image = FIXME
    # Send image to correct device
    image = FIXME
    # Make prediction
    output = FIXME
    # Find max index
    prediction = FIXME
    # Convert prediction to letter
    predicted_letter = FIXME
    # Return prediction
    return predicted_letter

In [ ]:
predict_letter("b.png")

Utilicemos también la función con la letra 'a' en el datset asl_images:

In [ ]:
predict_letter("a.png")

## 4b.5 Resumen

Buen trabajo con estos ejercicios. Has pasado por todo el proceso de entrenamiento de un modelo de alta precisión desde cero, y luego has utilizado el modelo para hacer nuevas y valiosas predicciones. Si tienes algo de tiempo, te animamos a que tomes fotos con tu webcam, las subas soltándolas en la carpeta data/asl_images, y pruebes el modelo con ellas. Para Mac puedes utilizar Photo Booth. Para Windows puedes seleccionar la aplicación Cámara en la pantalla de inicio. Esperamos que lo pruebes. Es una buena oportunidad para aprender algo de lenguaje de signos. Por ejemplo, prueba con las letras de tu nombre.

Podemos imaginar cómo se podría utilizar este modelo en una aplicación para enseñar a alguien el lenguaje de signos, o incluso ayudar a alguien que no puede hablar a interactuar con un ordenador.

# Preguntas

1. ¿Por qué es importante que las imágenes utilizadas para hacer predicciones tengan el mismo formato (dimensiones y color) que las imágenes con las que se entrenó el modelo?

2. Describe los pasos necesarios para procesar una imagen antes de hacer una predicción con el modelo entrenado. ¿Qué transformaciones se deben aplicar?

3. ¿Qué es la "dimensión del lote" (batch) y por qué es necesario agregarla cuando hacemos predicciones con el modelo?

4. ¿Cómo interpreta el modelo las predicciones que genera? Explica el formato de salida y cómo se convierte en una letra del alfabeto de lengua de signos.